This is a demo of how to use 'rok_estimator.py'.

In [1]:
from estimator.rok_estimator import *

In [ ]:
# The example below shows how to simulate the executions of individual RoKs.

# Specify parameters related to the underlying ring
ring_60 = RingParam(f=60)

# Create a new starting relation
rel = Relation(ring=ring_60,wdim=60,log_beta_wit_inf=0)
rel.show()

# Running pi_bdecomp reducing rel to rel_bdecomp
rel_bdecomp = rel.pi_bdecomp(ell=2)
rel_bdecomp.show(brief=True)

# Running pi_split reducing rel into rel_split
rel_split = rel.pi_split(2)
rel_split.show(brief=True)

# Running pi_batch reducing rel into rel_batch
rel_batch = rel.pi_batch()
rel_batch.show(brief=True)

# Running pi_fold reducing rel into rel_fold
rel_fold = rel.pi_fold(23)
rel_fold.show(brief=True)

# Running pi_norm reducing rel into rel_norm
rel_norm = rel.pi_norm()
rel_norm.show(brief=True)

Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 40, n_commit = 40, n_rel = 0
    wdim = 60, rep = 1
    log_2-norm (real | extr) = (8 |0 ), log_inf-norm (real | extr) = (0|0)
    wit size = 120.0 B 
 
Relation: wdim =       60, rep =   2, log_2-norm (real | extr) = (  8 |  0 ), log_inf-norm (real | extr) = (  0 |  0 ), wit size = 240.0 B , communication (growth | total) = (5.000 KB | 5.000 KB), soundness error (growth | total) = (2^-Infinity | 2^-Infinity)
Relation: wdim =       30, rep =   2, log_2-norm (real | extr) = (  8 |  0 ), log_inf-norm (real | extr) = (  0 |  0 ), wit size = 120.0 B , communication (growth | total) = (5.000 KB | 5.000 KB), soundness error (growth | total) = (2^-128 | 2^-128)
Relation: wdim =      

In [3]:
# The example below shows how to simulate the execution of a sequence of RoKs with specified parameters.

# Parameters for the underlying ring
ring_params = {
    "f": 60,
    "log_beta_sis_2": 40,
    "log_q": 64,
}

# Parameters for the starting relation
rel_params = {
    "wdim": 2**22,
    "rep": 25,
    "log_beta_wit_inf": 0,
}

# Parameters for the RoKs
ops_params = {
    "ell": 4,
    "d": 2,
}

# The sequence of RoKs
opener = [("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
loop = [("bdecomp", {"ell": ops_params["ell"]}), ("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
ops = opener + 10 * loop + [("finish", {})]

# Simulate the execution of the sequence of RoKs
sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show(brief=False)

Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^64, beta_sis_2: 2^40
    SIS module rank n_sis: 61, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 128
    |R_q| = 128.0 B, |R_q^(n_sis)| = 7.625 KB
 
Subtractive set parameters:
    cardinality: 12
    forward ell_2 expansion factor gamma_2: 2^0
    inverse ell_2 expansion factor theta_2: 2^4
    forward ell_inf expansion factor gamma_inf: 2^2
    inverse ell_inf expansion factor theta_inf: 2^5
 
Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 61, n_commit = 61, n_rel = 0
    wdim = 4194304, rep = 25
    log_2-norm (real | extr) = (16 |16 ), log_inf-norm (

In [4]:
# The example below shows how to define new RoKs and use them in simulation. 

class NewRelation(Relation):
    def execute(self, op, **kwargs):
        match op:
            case "new":
                return self.pi_new()
            case _:
                return super().execute(op, **kwargs)
            
    def pi_new(self):
        comm = 0 
        snd_err = 0 
        rel_params = {
            # "ring": self.ring,
            # "trivial": self.trivial,
            # "n_compress": self.n_compress,
            # "n_commit": self.n_commit,
            # "n_rel": self.n_rel,
            # "wdim": self.wdim,
            # "rep": self.rep,
            # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
            # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
            # "log_beta_wit_inf": self.log_beta_wit_inf
            "comm" : comm,      
            "acc_comm" : self.acc_comm + comm,       
            "snd_err" : snd_err,
            "acc_snd_err" : self.acc_snd_err + snd_err,           
            # "log_beta_ext_2_func" : lambda x : x, 
            # "log_beta_ext_inf_func" : lambda x : x, 
        }      
        return replace(self, **rel_params)
    
ring_params = {
    "f": 60,
    "log_beta_sis_2": 36,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**16 * 41,
    "rep": 25,
    "log_beta_wit_inf": 0,
}

ops = [("new", {}), ("finish", {})]

sim = Simulation(ring_params, rel_params, NewRelation)
sim.execute(ops)
sim.extract()
sim.show(brief=False)

Ring parameters:
    conductor f: 60, degree phi: 16, modulus q: 2^64, beta_sis_2: 2^36
    SIS module rank n_sis: 50, target SIS security: 128, resulting SIS security: 129
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 128
    |R_q| = 128.0 B, |R_q^(n_sis)| = 6.250 KB
 
Subtractive set parameters:
    cardinality: 12
    forward ell_2 expansion factor gamma_2: 2^0
    inverse ell_2 expansion factor theta_2: 2^4
    forward ell_inf expansion factor gamma_inf: 2^2
    inverse ell_inf expansion factor theta_inf: 2^5
 
Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 50, n_commit = 50, n_rel = 0
    wdim = 2686976, rep = 25
    log_2-norm (real | extr) = (16 |16 ), log_inf-norm (